#대회
[월간 데이콘 TV 손동작 제어 인식 AI 경진대회](https://dacon.io/competitions/official/236050/codeshare/7371?page=1&dtype=recent)

[주제]
TV를 제어하는 사용자의 손동작을 분류하는 AI 모델 개발

[설명]
TV를 제어하는 사용자의 손동작을 분류하기 위해서 30프레임의 1초 분량의 동영상(mp4)들이 입력 데이터로 주어지며,

동영상을 입력으로 받아 사용자의 손동작을 5가지의 Class로 분류하는 AI 모델을 개발해야합니다.



Class 0 : 스마트 TV 볼륨을 높입니다.

Class 1 : 스마트 TV 볼륨을 낮춥니다.

Class 2 : 스마트 TV의 재생 영상을 10초 전으로 점프합니다.

Class 3 : 스마트 TV의 재생 영상을 10초 앞으로 점프합니다.

Class 4 : 스마트 TV의 재생 영상을 중지합니다.


In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!unzip -qq "/content/gdrive/MyDrive/open (4).zip"

#import 

In [3]:
import random
import pandas as pd
import numpy as np
import os
import cv2

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torchvision.models as models

from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings(action='ignore') 

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#Hyperparameter Setting

In [5]:
CFG = {
    'FPS':30,
    'IMG_SIZE':128,
    'EPOCHS':10,
    'LEARNING_RATE':3e-4,
    'BATCH_SIZE':4,
    'SEED':41
}

#Fixed RandomSeed

In [6]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

#Data Load

In [7]:
df = pd.read_csv('./train.csv')

In [9]:
df

,id,path,label
0,TRAIN_000,./train/TRAIN_000.mp4,3
1,TRAIN_001,./train/TRAIN_001.mp4,0
2,TRAIN_002,./train/TRAIN_002.mp4,1
3,TRAIN_003,./train/TRAIN_003.mp4,4
4,TRAIN_004,./train/TRAIN_004.mp4,4
...,...,...,...
605,TRAIN_605,./train/TRAIN_605.mp4,0
606,TRAIN_606,./train/TRAIN_606.mp4,2
607,TRAIN_607,./train/TRAIN_607.mp4,1
608,TRAIN_608,./train/TRAIN_608.mp4,4


#Train / Validation Split

In [8]:
train, val, _, _ = train_test_split(df, df['label'], test_size=0.2, random_state=CFG['SEED'])

#CustomDataset

In [10]:
class CustomDataset(Dataset):
    def __init__(self,video_path_list,label_list):
      self.video_path_list = video_path_list
      self.label_list = label_list
    def __getitem__(self, index):
      frames=self.get_video(self.video_path_list[index])

      if self.label_list is not None:
        label = self.label_list[index]
        return frames, label
      else:
        return frames

    def __len__(self):
      return len(self.video_path_list)

    def get_video(self,path):
      frames =[]
      cap = cv2.VideoCapture(path)
      for _ in range(CFG['FPS']):
          _,img = cap.read()
          img = cv2.resize(img, (CFG['IMG_SIZE'], CFG['IMG_SIZE']))
          img = img / 255
          frames.append(img)
      return torch.FloatTensor(np.array(frames)).permute(3,0,1,2)

In [12]:
train_dataset = CustomDataset(train['path'].values, train['label'].values)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

In [13]:
val_dataset = CustomDataset(val['path'].values, val['label'].values)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

 num_class = 정답(label) 의 클래스 갯수

In [16]:
class BaseModel(nn.Module):
  def __init__(self,num_classes=5):
    super(BaseModel, self).__init__()
    self.feature_extract = nn.Sequential(
        nn.Conv3d(3,8,(3,3,3)),
        nn.ReLU(),
        nn.BatchNorm3d(8),
        nn.MaxPool3d(2),
        nn.Conv3d(8,32,(2,2,2)),
        nn.ReLU(),
        nn.BatchNorm3d(32),
        nn.MaxPool3d(2),
        nn.Conv3d(32,64,(2,2,2)),
        nn.ReLU(),
        nn.BatchNorm3d(64),
        nn.MaxPool3d(2),
        nn.Conv3d(64,128,(2,2,2)),
        nn.ReLU(),
        nn.BatchNorm3d(128),
        nn.MaxPool3d((1,7,7))
    )
    self.classifier = nn.Linear(512,num_classes)

  def forward(self,x):
    batch_size=x.size(0)
    x = self.feature_extract(x)
    x = x.view(batch_size,-1)
    x = self.classifier(x)
    return x


#Train

iter(호출가능한객체, 반복을끝낼값)

to(device) = gpu 할당을 선언

In [19]:
from sklearn.utils import validation
def train(model,optimizer, train_loader, val_loader, scheduler, device):
  model.to(device)
  criterion = nn.CrossEntropyLoss().to(device)

  best_val_score = 0
  best_model = None

  for epoch in range(1,CFG['EPOCHS']+1):
    model.train()
    train_loss = []
    for videos, labels in tqdm(iter(train_loader)):
        videos = videos.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        output = model(videos)
        #걍 파이토치는 loss를 이렇게 두는 듯. 일단 외워라. 이전에 분류기도 이렇게 코드를 짬.
        loss = criterion(output, labels)

        loss.backward()
        #optimizer.step() 을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정
        optimizer.step()
        #loss.item() 으로 손실이 갖고 있는 스칼라 값을 가져올 수 있습니다
        train_loss.append(loss.item())

    _val_loss,_val_score = validation(model,criterion,val_loader,device)
    _train_loss = np.mean(train_loss)
    print(f'Epoch [{epoch}], Train Loss : [{_train_loss:.5f}] Val Loss : [{_val_loss:.5f}] Val F1 : [{_val_score:.5f}]')

    if scheduler is not None:
      scheduler.step(_val_score)

    if best_val_score < _val_score:
      best_val_score = _val_score
      best_model = model

  return best_model

      


처음부터 끝까지 같은 learning rate를 사용할 수도 있지만, 학습과정에서 learning rate를 조정하는 learning rate scheduler를 사용할 수도 있다. 처음엔 큰 learning rate(보폭)으로 빠르게 optimize를 하고 최적값에 가까워질수록 learning rate(보폭)를 줄여 미세조정을 하는 것이 학습이 잘된다고 알려져있다. learning rate를 decay하는 방법이외에도 learning rate를 줄였다 늘렸다 하는 것이 더 성능향상에 도움이 된다는 연구결과도 있다. 

optimizer와 scheduler를 먼저 정의한 후, 학습할 때 batch마다 optimizer.step() 하고 epoch마다 scheduler.step()을 해주면 된다. 대략적인 코드를 작성하면 아래와 같은 흐름이다.

In [20]:
def validation(model,criterion,val_loader,device):
  model.eval()
  val_loss = []
  preds, trues = [], []

  with torch.no_grad():
    for videos, labels in tqdm(iter(val_loader)):
      videos = videos.to(device)
      labels = labels.to(device)

      logit = model(videos)

      loss = criterion(logit, labels)

      val_loss.append(loss.item())

      preds += logit.argmax(1).detach().cpu().numpy().tolist()
      trues += labels.detach().cpu().numpy().tolist()

    _val_loss = np.mean(val_loss)

  _val_score = f1_score(trues, preds, average='macro')

  return _val_loss, _val_score



Python numpy : tolist (array를 list로 변환. array list)

In [21]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2,threshold_mode='abs',min_lr=1e-8, verbose=True)

infer_model = train(model, optimizer, train_loader, val_loader, scheduler, device)

  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch [1], Train Loss : [1.53331] Val Loss : [1.33194] Val F1 : [0.45483]


  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

...

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch [9], Train Loss : [0.05146] Val Loss : [0.83341] Val F1 : [0.71910]


  0%|          | 0/122 [00:00<?, ?it/s]

  0%|          | 0/31 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.04453] Val Loss : [0.90657] Val F1 : [0.71732]


In [22]:
test = pd.read_csv('./test.csv')

In [23]:
test_dataset = CustomDataset(test['path'].values, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [24]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    preds = []
    with torch.no_grad():
        for videos in tqdm(iter(test_loader)):
            videos = videos.to(device)
            
            logit = model(videos)

            preds += logit.argmax(1).detach().cpu().numpy().tolist()
    return preds

In [25]:
preds = inference(model, test_loader, device)

  0%|          | 0/39 [00:00<?, ?it/s]